# Interactive Visualization

Topics included in this notebook:

* [Using Ibis and Vega to build interactive Dashboards](#ibis_vega)
* [Using Ibis and Holoviz to build interactive Dashboard](#ibis_holoviz)
* [Advanced Vega Demo 1 - Combining Tables: Comparing with Political Contributions](#adv_vega_1)
* [Advanced Vega Demo 2 - Replicating the MapD charting example with Altair](#adv_vega_2)
* [Advanced Holoviz Demo 1 - Blah](#adv_holoviz_1)
* [Advanced Holoviz Demo 2 - Blah](#adv_holoviz_2)

<a id='ibis_vega'></a>
## Using Ibis and Vega to build interactive Dashboards

In this example, we will extract out the [transformations](https://vega.github.io/vega-lite/docs/transform.html) in the Vega Lite spec generated by Altair and push them to the server by adding them to the SQL query with Ibis.

In [ ]:
import altair as alt
import ibis.omniscidb
import IPython.display
import ibis_vega_transform

First we connect to the table using Ibis:

In [ ]:
conn = ibis.omniscidb.connect(
    host='metis.mapd.com', user='demouser', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)
t = conn.table("flights_donotmodify")

First let's look at the average departure delays using a heatmap to color individual days of the year. We compose the Altair chart using an ibis expression.

In [ ]:
delay_by_month = alt.Chart(t[t.flight_dayofmonth, t.flight_month, t.depdelay]).mark_rect().encode(
    x='flight_dayofmonth:O',
    y='flight_month:O',
    color='average(depdelay)'
)
delay_by_month

We can take this a step further by adding widgets to extract and transform the data before visualizatiton. The transformations are pushed to the SQL statement executed by Ibis. 

We'll use a slider widget to select the month we'd like to visualize.

In [ ]:
slider = alt.binding_range(min=1, max=12, step=1)
select_month = alt.selection_single(fields=['flight_month'],
                                   bind=slider, init={'flight_month': 1})

alt.Chart(t[t.flight_dayofmonth, t.depdelay, t.flight_month]).mark_line().encode(
    x='flight_dayofmonth:O',
    y='average(depdelay)'
).add_selection(
    select_month
).transform_filter(
    select_month
)

The only data loaded into the browser for this chart are the individual points you see here in the line plot. 

<a id='ibis_holoviz'></a>
## Using Ibis and Holoviz to build interactive Dashboard

<a id='adv_vega_1'></a>
## Advanced Vega Demo 1 - Combining Tables: Comparing with Political Contributions

Now let's make a graph that uses data from two tables, "contributions_donotmodify" and "flights_donotmodify". Let's see how average delay time for flights from a state relates to the average political contribution size.

In [ ]:
t_contributions = conn.table("contributions_donotmodify")

Create the individual charts

In [ ]:
amount_by_state = alt.Chart(
    t_contributions[["contributor_state", "amount"]]
).mark_circle().encode(
    x="contributor_state",
    y=alt.Y("mean(amount):Q", axis=alt.Axis(grid=False)),
    color=alt.Color(value="green"),
    size="count()"
)

delay_by_state = alt.Chart(
    t[["origin_state", "depdelay"]]
).mark_square().encode(
    x="origin_state",
    y=alt.Y(
        "mean(depdelay):Q",
        axis=alt.Axis(grid=False)
    ),
    color=alt.Color(value="firebrick"),
    size="count()"
)

Overlay the two charts

In [ ]:
combined = (amount_by_state + delay_by_state).resolve_scale(y='independent', size='independent')
combined

<a id='adv_vega_2'></a> 
## Advanced Vega Demo 2 - Replicating the MapD charting example with Altair

Let's see if we can replicate [this](https://omnisci.github.io/mapd-charting/example/example1.html) MapD charting example in Python with Altair, Vega Lite, and Vega:

In [ ]:
states = alt.selection_multi(fields=['origin_state'])
airlines = alt.selection_multi(fields=['carrier_name'])

dates = alt.selection_interval(
    fields=['dep_timestamp'],
    encodings=['x'],
)

HEIGHT = 800
WIDTH = 1000

count_filter = alt.Chart(
    t[t.dep_timestamp, t.depdelay, t.origin_state, t.carrier_name],
    title="Selected Rows"
).transform_filter(
    airlines
).transform_filter(
    dates
).transform_filter(
    states
).mark_text().encode(
    text='count()'
)

count_total = alt.Chart(
    t,
    title="Total Rows"
).mark_text().encode(
    text='count()'
)

flights_by_state = alt.Chart(
    t[t.origin_state, t.carrier_name, t.dep_timestamp],
    title="Total Number of Flights by State"
).transform_filter(
    airlines
).transform_filter(
    dates
).mark_bar().encode(
    x='count()',
    y=alt.Y('origin_state', sort=alt.Sort(encoding='x', order='descending')),
    color=alt.condition(states, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).add_selection(
    states
).properties(
    height= 2 * HEIGHT / 3,
    width=WIDTH / 2
) + alt.Chart(
    t[t.origin_state, t.carrier_name, t.dep_timestamp],
).transform_filter(
    airlines
).transform_filter(
    dates
).mark_text(dx=20).encode(
    x='count()',
    y=alt.Y('origin_state', sort=alt.Sort(encoding='x', order='descending')),
    text='count()'
).properties(
    height= 2 * HEIGHT / 3,
    width=WIDTH / 2
)

carrier_delay = alt.Chart(
    t[t.depdelay, t.arrdelay, t.carrier_name, t.origin_state, t.dep_timestamp],
    title="Carrier Departure Delay by Arrival Delay (Minutes)"
).transform_filter(
    states
).transform_filter(
    dates
).transform_aggregate(
    depdelay='mean(depdelay)',
    arrdelay='mean(arrdelay)',
    groupby=["carrier_name"]
).mark_point(filled=True, size=200).encode(
    x='depdelay',
    y='arrdelay',
    color=alt.condition(airlines, alt.ColorValue("steelblue"), alt.ColorValue("grey")),
    tooltip=['carrier_name', 'depdelay', 'arrdelay']
).add_selection(
    airlines
).properties(
    height=2 * HEIGHT / 3,
    width=WIDTH / 2
) + alt.Chart(
    t[t.depdelay, t.arrdelay, t.carrier_name, t.origin_state, t.dep_timestamp],
).transform_filter(
    states
).transform_filter(
    dates
).transform_aggregate(
    depdelay='mean(depdelay)',
    arrdelay='mean(arrdelay)',
    groupby=["carrier_name"]
).mark_text().encode(
    x='depdelay',
    y='arrdelay',
    text='carrier_name',
).properties(
    height=2 * HEIGHT / 3,
    width=WIDTH / 2
)

time = alt.Chart(
    t[t.dep_timestamp, t.depdelay, t.origin_state, t.carrier_name],
    title='Number of Flights by Departure Time'
).transform_filter(
    'datum.dep_timestamp != null'
).transform_filter(
    airlines
).transform_filter(
    states
).mark_line().encode(
    alt.X(
        'yearmonthdate(dep_timestamp):T',
    ),
    alt.Y(
        'count():Q',
        scale=alt.Scale(zero=False)
    )
).add_selection(
    dates
).properties(
    height=HEIGHT / 3,
    width=WIDTH + 50
)

(
    (count_filter | count_total) & (flights_by_state | carrier_delay) & time
).configure_axis(
    grid=False
).configure_view(
    strokeOpacity=0
)

<a id='adv_holoviz_1'></a>
## Advanced Holoviz Demo 1 - Blah

<a id='adv_holoviz_2'></a>
## Advanced Holoviz Demo 2 - Blah